# HOOMD FFS API Test - April 28th 2022

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from numba import njit, vectorize, float32

from monk import nb

from typing import Callable, Optional

import hoomd
import hoomd.forward_flux.forward_flux as ffs
import gsd.hoomd

from monk import prep
from monk import pair
import ex_render

In [ ]:
cpu = hoomd.device.CPU()
seed = 1234
# sim = ffs.ForwardFluxSimulation(cpu, 3, seed=seed)
sim = hoomd.Simulation(cpu, seed=seed)

In [ ]:
N = 512
rng = prep.init_rng(seed + 1)
L = prep.len_from_phi(N, 1.2)
snap = prep.approx_euclidean_snapshot(N, L, rng, dim=3, ratios=[80,20], diams=[1.0, 0.6])

sim.create_state_from_snapshot(snap)

In [ ]:
ex_render.render_sphere_frame(frame=sim.state.get_snapshot())

In [ ]:
sim.create_state_from_gsd("test-system.gsd")

In [ ]:
integrator = hoomd.md.Integrator(dt=0.0025)
tree = hoomd.md.nlist.Tree(0.3)
# lj = pair.KA_LJ_DPD(tree, 1.5)
lj = pair.KA_LJ(tree)

var = hoomd.variant.Ramp(1.5, 0.4, sim.timestep, sim.timestep + 1_000_000)
lang = hoomd.md.methods.Langevin(hoomd.filter.All(), var)
# lang = hoomd.md.methods.NVE(hoomd.filter.All())
integrator.forces = [lj]
integrator.methods = [lang]

sim.operations.integrator = integrator

In [ ]:
sim.run(10_000)
ex_render.render_sphere_frame(frame=sim.state.get_snapshot())

In [ ]:
sim.run(1_100_000)
hoomd.write.GSD.write(sim.state, "equil-system-T0.4.gsd")

In [ ]:
integrator = hoomd.md.Integrator(dt=0.0025)
tree = hoomd.md.nlist.Tree(0.3)
# lj = pair.KA_LJ_DPD(tree, 1.5)
lj = pair.KA_LJ(tree)

var = hoomd.variant.Ramp(0.4, 0.1, sim.timestep, sim.timestep + 1_000_000)
lang = hoomd.md.methods.Langevin(hoomd.filter.All(), var)
# lang = hoomd.md.methods.NVE(hoomd.filter.All())
integrator.forces = [lj]
integrator.methods = [lang]

sim.operations.integrator = integrator

In [ ]:
sim.run(1_100_000)
hoomd.write.GSD.write(sim.state, "equil-system-T0.1.gsd")

In [ ]:
lj.kT = 0.1

In [ ]:
lang.kT = 0.6

In [ ]:
sim.run(10_000)
ex_render.render_sphere_frame(frame=sim.state.get_snapshot())

In [ ]:
lang.kT = 0.01

In [ ]:
sim.run(100_000)
ex_render.render_sphere_frame(frame=sim.state.get_snapshot())

In [ ]:
sim.run(10_000)
ex_render.render_sphere_frame(frame=sim.state.get_snapshot())

In [ ]:
hoomd.write.GSD.write(sim.state, "test-system.gsd")

In [ ]:
sim.operations.writers.clear()
basin_op = sim.sample_basin(100_000, 10)

In [ ]:
plt.hist(basin_op, bins=25)

In [ ]:
plt.plot(basin_op)

In [ ]:
plt.plot(basin_op)

In [ ]:
plt.plot(basin_op)

In [ ]:
plt.plot(basin_op)

In [ ]:
writer = hoomd.write.GSD(trigger=hoomd.trigger.Periodic(10), filename="test.gsd", mode="xb", filter=hoomd.filter.All())
sim.operations.writers.clear()
sim.operations.writers.append(writer)
sim.run(10_000)

In [ ]:
sim.run(10_000, write_at_start=True)

In [ ]:
ex_render.render_sphere_frame(frame=sim.state.get_snapshot())

In [ ]:
integrator, langevin, forces = sim._assert_langevin()

In [ ]:

fire = hoomd.md.minimize.FIRE(**sim.fire_kwargs)
nve = hoomd.md.methods.NVE(hoomd.filter.All())
fire.methods = [nve]
fire.forces = [forces.pop() for _ in range(len(forces))]


In [ ]:
len(fire.forces)

In [ ]:
sim.operations.integrator = fire

In [ ]:
fire.converged

In [ ]:
sim.run(1000)

In [ ]:
fire.converged